In [1]:
#Inportar Programas#
import bempp.api
import numpy as np 
import dolfin
from readoff import *
from readpqr import *
import trimesh
import time
start = time.time()

In [2]:
#Datos Principales. 
em = 4.     #[-] Permitividad electrica interior.
ei = 10.    #[-] Permitividad electrica de la capa intermedia.
es = 80.    #[-] Permitividad electrica exterior.
k = 0.125   #[1/A] Inverso de la longitud de Debey-Huckel del fluido.

#Ensablaje de los operadores de frontera: #'fmm' para moleculas con un gran mayor numero de vertices.
#'default_nonlocal' para moleculas con un pequeño numero de vertices.
Assemble = 'default_nonlocal' 

In [3]:
#Datos de la posicion en el espacio, carga y radios de los atomos de la molecula.
PC,Q,R = readpqr('PQR/Sphere5Q3.pqr')  #Eleccion del "pqr" de la molecula

In [4]:
#Importar la malla volumetrica del cascaron de la molecula
from dolfin import Mesh
mesh = Mesh("Mallas_V/outputTetMeshSphere56R0.xml") #Creacion de la malla volumetrica

In [5]:
#Generar espacios de funcionales globales del potencial en Fem y su derivada en Bem
from bempp.api.external import fenics

fenics_space = dolfin.FunctionSpace(mesh, "CG", 1) #Potencial electrostatico en la interfaz y dominio del soluto
trace_space, trace_matrix = \
    fenics.fenics_to_bempp_trace_data(fenics_space) # Espacio de la traza global para trabajar en BEM y FEM simultaneamente

In [6]:
#Proceso para separar el trace_space para el caso de la malla inferior y la malla superior de forma individual

#Codigo para identificar vertices y caras de la malla superior e inferior
faces0 = trace_space.grid.elements
vertices0 = trace_space.grid.vertices
meshSP = trimesh.Trimesh(vertices = vertices0.transpose(), faces= faces0.transpose())
mesh_split = meshSP.split()
print("Found %i meshes"%len(mesh_split))
V1 = len(mesh_split[0].vertices)
F1 = len(mesh_split[0].faces)

#Obtencion de la malla superficial inferior
faces1 = faces0.transpose()[:F1]
vertices1 = vertices0.transpose()[:V1]
grid1 = bempp.api.grid.grid.Grid(vertices1.transpose(), faces1.transpose())
bempp_space1 = bempp.api.function_space(grid1, "DP", 0) #Derivada del potencial electrostatico en la interfaz inferior
trace_space1 = bempp.api.function_space(grid1, "P", 1) # Espacio de la traza para trabajar en BEM inferior y FEM simultaneamente.

#Obtencion de la malla superficial superior
faces2 = faces0.transpose()[F1:]
vertices2 = vertices0.transpose()[V1:]
grid2 = bempp.api.grid.grid.Grid(vertices2.transpose(), (faces2-len(vertices1)).transpose())
bempp_space2 = bempp.api.function_space(grid2, "DP", 0) #Derivada del potencial electrostatico en la interfaz superior
trace_space2 = bempp.api.function_space(grid2, "P", 1) # Espacio de la traza para trabajar en BEM superior y FEM simultaneamente.

#Visualizacion de elementos
print("FEM dofs: {0}".format(mesh.num_vertices()))
print("BEM1 dofs: {0}".format(bempp_space1.global_dof_count))
print("BEM2 dofs: {0}".format(bempp_space2.global_dof_count))
print("Tra1 dofs: {0}".format(trace_space1.global_dof_count))
print("Tra2 dofs: {0}".format(trace_space2.global_dof_count))
print("TraL dofs: {0}".format(trace_space.global_dof_count))

Found 2 meshes
FEM dofs: 707
BEM1 dofs: 576
BEM2 dofs: 830
Tra1 dofs: 290
Tra2 dofs: 417
TraL dofs: 707


In [7]:
#Proceso para separar el trace_matrix para el caso de la malla inferior y la malla superior de forma individual
Nodos = np.zeros(trace_space.global_dof_count)
Lista_Vertices = []

#Procedimeito para ubicar los vertices del trace inferior en trace global
for i in range(len(trace_space1.grid.vertices.T)):
    valores = np.linalg.norm(trace_space1.grid.vertices[:, i] - trace_space.grid.vertices.T,axis= 1)
    index = np.argmin(valores)
    Lista_Vertices.append(index)
    
Nodos[Lista_Vertices] = 1
trace_matrix1 = trace_matrix[Nodos.astype(bool)]
trace_matrix2 = trace_matrix[np.logical_not(Nodos)]

In [8]:
#Generar los operadores de frontera
bempp.api.GLOBAL_PARAMETERS.fmm.expansion_order = 5
#Operadores de identidad
I1 = bempp.api.operators.boundary.sparse.identity(trace_space1, bempp_space1, bempp_space1) # 1
I2 = bempp.api.operators.boundary.sparse.identity(trace_space2, bempp_space2, bempp_space2) # 1
mass1 = bempp.api.operators.boundary.sparse.identity(bempp_space1, trace_space1, trace_space1) # 1
mass2 = bempp.api.operators.boundary.sparse.identity(bempp_space2, trace_space2, trace_space2) # 1

#Dominio del soluto Ωm en BEM
K1 = bempp.api.operators.boundary.laplace.double_layer(trace_space1, bempp_space1, bempp_space1, assembler=Assemble) #K
V1 = bempp.api.operators.boundary.laplace.single_layer(bempp_space1, bempp_space1, bempp_space1, assembler=Assemble) #V
Z1 = bempp.api.ZeroBoundaryOperator(bempp_space2, bempp_space1, bempp_space1) #0

#Dominio del solvente Ωs en BEM
Z2 = bempp.api.ZeroBoundaryOperator(bempp_space1, bempp_space2, bempp_space2) #0
if k==0:
    K2 = bempp.api.operators.boundary.laplace.double_layer(trace_space2, bempp_space2, bempp_space2, assembler=Assemble) #K
    V2 = bempp.api.operators.boundary.laplace.single_layer(bempp_space2, bempp_space2, bempp_space2, assembler=Assemble) #V  
else:
    K2 = bempp.api.operators.boundary.modified_helmholtz.double_layer(trace_space2, bempp_space2, bempp_space2, k, assembler=Assemble) #K
    V2 = bempp.api.operators.boundary.modified_helmholtz.single_layer(bempp_space2, bempp_space2, bempp_space2, k, assembler=Assemble) #V

In [9]:
#Definir el espacio funcional de Dolfin
u = dolfin.TrialFunction(fenics_space)
v = dolfin.TestFunction(fenics_space)

In [10]:
#Creacion de funcion del potencial de Coulomb   
@bempp.api.complex_callable(jit=False)
def U_c(x, n, domain_index, result):
    global Q,PC,em,S1
    result[:] = (1 / (4.*np.pi*em))  * np.sum( Q / np.linalg.norm( x - PC, axis=1))
Uca = bempp.api.GridFunction(bempp_space1, fun=U_c)

#Construccion del vector derecho
# Rhs en Ωm en BEM
rhs_bem1 = (Uca).projections(bempp_space1)
# Rhs en Ωi en FEM
rhs_fem =  np.zeros(mesh.num_vertices()) 
# Rhs en Ωs en BEM
rhs_bem2 = np.zeros(bempp_space2.global_dof_count) 
# La combinacion de rhs
rhs = np.concatenate([rhs_bem1, rhs_fem, rhs_bem2])

In [11]:
#Construccion matriz izquierda 3x3
from bempp.api.external.fenics import FenicsOperator
from scipy.sparse.linalg import LinearOperator
from bempp.api.assembly.blocked_operator import BlockedDiscreteOperator
blocks = [[None,None,None],[None,None,None],[None,None,None]]

trace_op1 = LinearOperator(trace_matrix1.shape, lambda x:trace_matrix1*x)
trace_op2 = LinearOperator(trace_matrix2.shape, lambda x:trace_matrix2*x)
A = FenicsOperator(ei*(dolfin.inner(dolfin.nabla_grad(u),
                                 dolfin.nabla_grad(v)) ) * dolfin.dx)
#Posicion de la matriz 3x3
blocks[0][0] = V1.weak_form()                     # -V
blocks[0][1] = (0.5*I1-K1).weak_form()*trace_op1  # 0.5+K
blocks[0][2] = Z1.weak_form()                     # 0
 
blocks[1][0] = -trace_matrix1.T*em*mass1.weak_form().A   # -Mg1
blocks[1][1] =  A.weak_form()                            # A
blocks[1][2] = -trace_matrix2.T*es*mass2.weak_form().A   # -Mg2
 
blocks[2][0] = Z2.weak_form()                     # 0
blocks[2][1] = (0.5*I2-K2).weak_form()*trace_op2  # 0.5-K
blocks[2][2] = V2.weak_form()                     # V

blocked = bempp.api.assembly.blocked_operator.BlockedDiscreteOperator(np.array(blocks)) 

In [12]:
#Creacion del precondicionador Matriz de Masa para BEM/FEM/BEM
from bempp.api.assembly.discrete_boundary_operator import InverseSparseDiscreteBoundaryOperator
from scipy.sparse.linalg import LinearOperator
from scipy.sparse import diags

P2 = diags(1./(blocked[1,1].A).diagonal())
P1 = InverseSparseDiscreteBoundaryOperator(
    bempp.api.operators.boundary.sparse.identity(
        bempp_space1, bempp_space1, bempp_space1).weak_form())
P3 = InverseSparseDiscreteBoundaryOperator(
    bempp.api.operators.boundary.sparse.identity(
        bempp_space2, bempp_space2, bempp_space2).weak_form())

def apply_prec(x):
    """Apply the block diagonal preconditioner"""
    m1 = P1.shape[0]
    m2 = P2.shape[0]
    m3 = P3.shape[0]
    n1 = P1.shape[1]
    n2 = P2.shape[1]
    n3 = P3.shape[1]
 
    res1 = P1.dot(x[:n1])
    res2 = P2.dot(x[n1: n1+n2])
    res3 = P3.dot(x[n1+n2:])
    return np.concatenate([res1, res2, res3])

p_shape = (P1.shape[0] + P2.shape[0] + P3.shape[0], P1.shape[1] + P2.shape[1] + P3.shape[1])
P = LinearOperator(p_shape, apply_prec, dtype=np.dtype('complex128'))

/home/mauricioguerrero/Software/miniconda3/envs/bempp_prod/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [13]:
#Se resuelve la solucion de la Ecuacion matricial Ax=B
#Contador de iteraciones
it_count = 0
def count_iterations(x):
    global it_count
    it_count += 1
    if (it_count / 20000) == (it_count // 20000):
        print(it_count,x)

# Solucion por GMRES
from scipy.sparse.linalg import gmres
start1 = time.time()
soln, info = gmres(blocked, rhs, M=P, callback=count_iterations,tol=1e-6)  
end1 = time.time() 

# Tiempo de resolver la ecuacion
curr_time1 = (end1 - start1) 
print("Numero de iteraciones de GMRES: {0}".format(it_count))
print("Tiempo total en GMRES: {:5.2f} [s]".format(curr_time1)) 

soln_bem1 = soln[:bempp_space1.global_dof_count] 
soln_fem  = soln[bempp_space1.global_dof_count : bempp_space1.global_dof_count + mesh.num_vertices()]
soln_bem2 = soln[bempp_space1.global_dof_count + mesh.num_vertices():]

Numero de iteraciones de GMRES: 103
Tiempo total en GMRES:  0.26 [s]


In [14]:
#Calcula todo el dominio global del potencial a partir de la solucion de los borde de ambas interfaces.
#(Incluido la funcion si se quiere calcular el valor de los tres dominios)

# Calcula la solucion del potencial real en el dominio de FEM en la region intermedia 
u = dolfin.Function(fenics_space)
u.vector()[:] = np.ascontiguousarray(np.real(soln_fem)) 

# Solucion para datos de Dirichlet en la interfaz inferior
dirichlet_data1 = trace_matrix1 * soln_fem
dirichlet_fun1 = bempp.api.GridFunction(trace_space1, coefficients=dirichlet_data1)
# Solucion para datos de Neumann en la interfaz inferior
neumann_fun1 = bempp.api.GridFunction(bempp_space1, coefficients=soln_bem1)

# Solucion para datos de Dirichlet en la interfaz superior
dirichlet_data2 = trace_matrix2 * soln_fem
dirichlet_fun2 = bempp.api.GridFunction(trace_space2, coefficients=dirichlet_data2)
# Solucion para datos de Neumann en la interfaz superior
neumann_fun2 = bempp.api.GridFunction(bempp_space2, coefficients=soln_bem2)

In [15]:
#Calculo del potencial en la posicion de los atomos de la molecula
VF1 = bempp.api.operators.potential.laplace.single_layer(bempp_space1, np.transpose(PC)) 
KF1 = bempp.api.operators.potential.laplace.double_layer(trace_space1, np.transpose(PC))
uF = -VF1*neumann_fun1 + KF1*dirichlet_fun1 

#Resultado de la energia de solvatacion por atomo
for i in range(len(PC)):
    Sum1 = (uF[0][i]).real*Q[i]
    Ei = 0.5*4.*np.pi*332.064*(Sum1)
    print(Ei)
    
#Resultado de la energia de solvatacion total  
E_Solv = 0.5*4.*np.pi*332.064*np.sum(Q*uF).real 
print('Energia de Solvatacion: {:7.6f} [kCal/mol]'.format(E_Solv) )

end = time.time()
curr_time = (end - start)   
print("Tiempo total: {:5.2f} [s]".format(curr_time))

-22.818177900080315
-22.819659440147923
-22.819185824713674
Energia de Solvatacion: -68.457023 [kCal/mol]
Tiempo total: 24.97 [s]
